# Random forest + temporal models

In the previous section, we loaded and processed the data for one participant,
used it to train a classifier, and tested the model on another participant.

In this section, let's make use of all the data from the 151 participants.
We will train the classifier on data from 100 participants, and set aside 51
participants for testing.

Finally, we will explore ways to account for the temporal dependency such as
mode smoothing and hidden Markov models.

## Setup

In [ ]:
import os
import numpy as np
import pandas as pd
from glob import glob
import scipy.stats as stats
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from joblib import Parallel, delayed
import urllib
import shutil
from tqdm.auto import tqdm
import utils  # helper functions -- check out utils.py
import zipfile

# For reproducibility
np.random.seed(42)

## Process dataset

Use the following script to process the whole Capture-24 dataset.

*Note*: Alternatively, you can download the processed files by uncommenting the last paragraph in the cell below. 
We recommend this option for devices with <16 GB of RAM.  In any case, make
sure you understand the data processing below.

*This takes up to 15 minutes*

In [ ]:
def load_all_and_make_windows(datafiles, N=999):

    def worker(datafile):
        X, Y, T = utils.make_windows(utils.load_data(datafile), winsec=30)
        pid = os.path.basename(datafile).split(".")[0]  # participant ID
        pid = np.asarray([pid] * len(X))
        return X, Y, T, pid

    results = Parallel(n_jobs=4)(
        delayed(worker)(datafile) for datafile in tqdm(datafiles[:N])
    )

    X = np.concatenate([result[0] for result in results])
    Y = np.concatenate([result[1] for result in results])
    T = np.concatenate([result[2] for result in results])
    pid = np.concatenate([result[3] for result in results])

    return X, Y, T, pid

# ------------------------------------------
# Process all files
# ------------------------------------------
DATAFILES = 'capture24/P[0-9][0-9][0-9].csv.gz'
X, Y, T, pid = load_all_and_make_windows(glob(DATAFILES))
# Save arrays for future use
os.makedirs("processed_data/", exist_ok=True)
np.save("processed_data/X.npy", X)
np.save("processed_data/Y.npy", Y)
np.save("processed_data/T.npy", T)
np.save("processed_data/pid.npy", pid)

# -------------------------------------------------
# Uncomment below lines to instead download processed files
# -------------------------------------------------
# print(f"Downloading processed files...")
# os.makedirs("processed_data/", exist_ok=True)
# url = "https://wearables-files.ndph.ox.ac.uk/files/processed_data.zip"
# with urllib.request.urlopen(url) as f_src, open(f"processed_data.zip", "wb") as f_dst:
#     shutil.copyfileobj(f_src, f_dst)
# print(f"Finished downloading")

# print("Unzipping...")
# with zipfile.ZipFile("processed_data.zip", "r") as f:
#     f.extractall(".")
# print(f"Finished unzipping")

In [ ]:
# Load processed files
X = np.load('processed_data/X.npy', mmap_mode='r')
Y = np.load('processed_data/Y.npy')
T = np.load('processed_data/T.npy')
pid = np.load('processed_data/pid.npy')

**Exercise 1**: What is the meaning of each of the loaded processed files (`X`, `Y`, `T` and `pid`)? Hint: inspect the contents of each, and well as their shape.

**Exercise 2**: How many unique annotations are found in `Y`?

#### Extract features

In [ ]:
def extract_features(xyz):
    ''' Extract features. xyz is an array of shape (N,3) '''

    feats = {}
    feats['xMean'], feats['yMean'], feats['zMean'] = np.mean(xyz, axis=0)
    feats['xStd'], feats['yStd'], feats['zStd'] = np.std(xyz, axis=0)
    v = np.linalg.norm(xyz, axis=1)  # magnitude stream
    feats['mean'], feats['std'] = np.mean(v), np.std(v)

    return feats

# Extract features
X_feats = pd.DataFrame(Parallel(n_jobs=4)(delayed(extract_features)(x) for x in tqdm(X)))
X_feats.to_pickle('processed_data/X_feats.pkl')
print(X_feats)

**Exercise 3**: Adjust the function above to engineer your own features!

Note the time spent extracting features from all X segments. Is this necessary every time we wish to test our code?

#### Map annotations

As before, summarise the annotations by mapping to a reduced set of classes

In [ ]:
# As before, let's map the text annotations to simplified labels
anno_label_dict = pd.read_csv(
    "capture24/annotation-label-dictionary.csv", 
    index_col='annotation', 
    dtype='string'
)
Y = anno_label_dict.loc[Y, 'label:Willetts2018'].to_numpy()

print("Label districution (# windows)")
print(pd.Series(Y).value_counts())

**Exercise 4**: How many hours of "bicycling" in total do we have?

## Train/test split

In [ ]:
# Hold out participants P101-P151 for testing (51 participants)
test_ids = [f'P{i}' for i in range(101,152)]
mask_test = np.isin(pid, test_ids)
mask_train = ~mask_test
X_train, Y_train, T_train, pid_train = \
    X_feats[mask_train], Y[mask_train], T[mask_train], pid[mask_train]
X_test, Y_test, T_test, pid_test = \
    X_feats[mask_test], Y[mask_test], T[mask_test], pid[mask_test]
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

## Train a random forest classifier

*This may take a while*

In [ ]:
# Argument oob_score=True to be used for HMM smoothing (see later below)
clf = BalancedRandomForestClassifier(
    n_estimators=100,
    replacement=True,
    sampling_strategy='not minority',
    oob_score=True,
    n_jobs=4,
    random_state=42,
    verbose=1
)
clf.fit(X_train, Y_train)

**Exercise 5**: Play with the parameters of the classifier. See the [API reference](https://imbalanced-learn.org/stable/references/generated/imblearn.ensemble.BalancedRandomForestClassifier.html#imblearn.ensemble.BalancedRandomForestClassifier) for parameter descriptions.

## Model performance

In [ ]:
Y_test_pred = clf.predict(X_test)
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred))

Overall, the model seems to do well in distinguishing between very inactive
periods ("sit-stand" and "sleep") and very active ones ("bicycling"), but there
seems to be confusion between the remaining activities.

#### Plot predicted vs. true activity profiles

Using our utility function, let's plot the activity profile for one participant, e.g.
`P101`.

In [ ]:
mask = pid_test == 'P101'
fig, axs = utils.plot_compare(T_test[mask],
                              Y_test[mask],
                              Y_test_pred[mask],
                              trace=X_test.loc[mask, 'std'])
fig.show()

The profile plots look good at first glance. After all, the majority of
activities happen to be of the sedentary type for which the model performs
well &mdash; this is reflected by the relatively high `weighted avg` scores in
the table report.
However, the `macro avg` scores are still low, and we see that the model
struggles to classify relevant activities such as bicycling and walking.
Moreover, we often find discontinuities during the sleep periods. This is
because the model is only trained to classify each window instance independently
and does not account for temporal dependencies.

## Accounting for temporal dependency

### Rolling mode smoothing
Let's use rolling mode smoothing to smooth the model predictions: Pick the
most popular label within a rolling time window.


In [ ]:
def mode(alist):
    ''' Mode of a list, but return middle element if ambiguous '''
    m, c = stats.mode(alist)
    m, c = m.item(), c.item()
    if c==1:
        return alist[len(alist)//2]
    return m

def rolling_mode(t, y, window_size='100S'):
    y_dtype_orig = y.dtype
    # Hack to make it work with pandas.Series.rolling()
    y = pd.Series(y, index=t, dtype='category')
    y_code_smooth = y.cat.codes.rolling(window_size).apply(mode, raw=True).astype('int')
    y_smooth = pd.Categorical.from_codes(y_code_smooth, dtype=y.dtype)
    y_smooth = np.asarray(y_smooth, dtype=y_dtype_orig)
    return y_smooth

In [ ]:
# Smooth the predictions of each participant
Y_test_pred_smooth = []
unqP, indP = np.unique(pid_test, return_index=True)
unqP = unqP[np.argsort(indP)]  # keep the order or else we'll scramble our arrays
for p in unqP:
    mask = pid_test == p
    Y_test_pred_smooth.append(rolling_mode(T_test[mask], Y_test_pred[mask]))
Y_test_pred_smooth = np.concatenate(Y_test_pred_smooth)

print('\nClassifier performance -- mode smoothing')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred_smooth))

# Check again participant
mask = pid_test == 'P101'
fig, axs = utils.plot_compare(T_test[mask],
                              Y_test[mask],
                              Y_test_pred_smooth[mask],
                              trace=X_test.loc[mask, 'std'])
fig.show()

The simple mode smoothing already improved performance slightly.

### Hidden Markov Model

A more principled approch is to use a Hidden Markov Model (HMM). It can help
smooth the sequence of predictions and avoid weird sequences such as
"sleep"-"bicycling"-"sleep". 
Here the random forest predictions are interpreted as "observations" of the
"hidden ground truth". The emission matrix can be estimated from probabilistic
predictions of model, and the transition matrix can be estimated from the ground
truth sequence of activities. The prior probabilities can be set as the rates
observed in the dataset, or a uniform (uninformative) prior.

Check `utils.train_hmm` and `utils.viterbi` for implementationd details.

**Exercise 5**: How does random forest provide out-of-bag estimations for free?

**Exercise 6**: Why do we need out-of-bag estimations?

In [ ]:
# Use the conveniently provided out-of-bag probability predictions from the
# random forest training process.
Y_train_prob = clf.oob_decision_function_  # out-of-bag probability predictions
labels = clf.classes_  # need this to know the label order of cols of Y_train_prob
hmm_params = utils.train_hmm(Y_train_prob, Y_train, labels)  # obtain HMM matrices/params
Y_test_pred_hmm = utils.viterbi(Y_test_pred, hmm_params)  # smoothing
print('\nClassifier performance -- HMM smoothing')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred_hmm))

# Check again participant
mask = pid_test == 'P101'
fig, ax = utils.plot_compare(T_test[mask],
                             Y_test[mask],
                             Y_test_pred_hmm[mask],
                             trace=X_test.loc[mask, 'std'])
fig.show()

HMM further improves the performance scores.

## Is a simple logistic regression enough?

*This takes a while*

In [ ]:
clf_LR = LogisticRegression(
    max_iter=1000,
    multi_class='multinomial',
    random_state=42)
scaler = StandardScaler()
pipe = make_pipeline(scaler, clf_LR)
pipe.fit(X_train, Y_train)

Y_test_pred_LR = pipe.predict(X_test)

# HMM smoothing
Y_train_LR_prob = pipe.predict_proba(X_train)  # sorry! LR doesn't provide OOB estimates for free
labels = pipe.classes_
hmm_params_LR = utils.train_hmm(Y_train_LR_prob, Y_train, labels)
Y_test_pred_LR_hmm = utils.viterbi(Y_test_pred_LR, hmm_params_LR)  # smoothing

print('\nClassifier performance -- Logistic regression')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred_LR_hmm))

# Check again participant
mask = pid_test == 'P101'
fig, axs = utils.plot_compare(T_test[mask],
                      Y_test[mask],
                      Y_test_pred_LR_hmm[mask],
                      trace=X_test.loc[mask, 'std'])
fig.show()

The LR model performed well on the easier classes "sleep" and "sit-stand",
but was much worse on all the other classes.